**HOW TO CALL THE CLASS FROM YOUR NOTEBOOK**

In [8]:
from api.mapbox_api import MapboxAPI
import requests
from urllib.request import urlopen
import json
from shapely.geometry import shape, Polygon

In [2]:
api = MapboxAPI()

In [3]:
response = api.get_data((41.97557881655979, -87.71361314512697), 15)
response

{'coordinates': [[[-87.764613, 42.097237],
   [-87.764624, 42.095568],
   [-87.763706, 42.095486],
   [-87.763657, 42.094535],
   [-87.762796, 42.094396],
   [-87.762685, 42.092507],
   [-87.761896, 42.092296],
   [-87.761722, 42.09147],
   [-87.761001, 42.091191],
   [-87.760775, 42.089417],
   [-87.758967, 42.087579],
   [-87.759263, 42.081228],
   [-87.759613, 42.08089],
   [-87.761613, 42.081441],
   [-87.762613, 42.079932],
   [-87.764469, 42.079579],
   [-87.762613, 42.079455],
   [-87.761613, 42.078659],
   [-87.759491, 42.078701],
   [-87.759199, 42.071579],
   [-87.758144, 42.069048],
   [-87.757467, 42.068579],
   [-87.757211, 42.066981],
   [-87.756541, 42.066506],
   [-87.757021, 42.064986],
   [-87.758613, 42.064968],
   [-87.760613, 42.062974],
   [-87.761613, 42.063522],
   [-87.763613, 42.06281],
   [-87.764263, 42.062929],
   [-87.764613, 42.063621],
   [-87.765613, 42.06293],
   [-87.768064, 42.06303],
   [-87.769272, 42.062579],
   [-87.761199, 42.061993],
   [-87.76

1) Function to append a list of coordinates to a new column of information for each library
    - get library data
    - create new column
    - create function

In [4]:
#importing library data
import pandas as pd
import geopandas as gpd
branchLocations = gpd.read_file('../data/clean/branches.csv')

In [5]:
#Replacing Location with longitude and latitude columns
import ast
branchLocations[['LATITUDE', 'LONGITUDE']] = [ast.literal_eval(x)[:2] for x in branchLocations['LOCATION']]
branchLocations.loc[:, 'LATITUDE'] = pd.to_numeric(branchLocations.loc[:, 'LATITUDE'])
branchLocations.loc[:, 'LONGITUDE'] = pd.to_numeric(branchLocations.loc[:, 'LONGITUDE'])
branchLocations.head()

,BRANCH,HOURS OF OPERATION,ADDRESS,CITY,STATE,ZIP,PHONE,WEBSITE,LOCATION,PERMANENTLY_CLOSED,geometry,LATITUDE,LONGITUDE
0,Albany Park,"Mon. & Wed., 10-6; Tues. & Thurs., Noon-8; Fri...",3401 W. Foster Ave.,Chicago,IL,60625,(773) 539-5450,https://www.chipublib.org/locations/3/,"(41.97557881655979, -87.71361314512697)",0,None,41.975579,-87.713613
1,Altgeld,"Mon. & Wed., Noon-8; Tues. & Thurs., 10-6; Fri...",955 E. 131st Street,Chicago,IL,60827,(312) 747-3270,https://www.chipublib.org/locations/4/,"(41.65719847558056, -87.59883829075173)",0,None,41.657198,-87.598838
2,Archer Heights,"Mon. & Wed., Noon-8; Tues. & Thurs., 10-6; Fri...",5055 S. Archer Ave.,Chicago,IL,60632,(312) 747-9241,https://www.chipublib.org/locations/5/,"(41.80110836194246, -87.72648385568911)",0,None,41.801108,-87.726484
3,Austin,"Mon. & Wed., Noon-8; Tues. & Thurs., 10-6; Fri...",5615 W. Race Ave.,Chicago,IL,60644,(312) 746-5038,https://www.chipublib.org/locations/6/,"(41.88917312206642, -87.7658203582574)",0,None,41.889173,-87.765820
4,Austin-Irving,"Mon. & Wed., Noon-8; Tues. & Thurs., 10-6; Fri...",6100 W. Irving Park Rd.,Chicago,IL,60634,(312) 744-6222,https://www.chipublib.org/locations/7/,"(41.95312267684315, -87.77928489355646)",0,None,41.953123,-87.779285


In [80]:
 branchLocations.loc[0,'15 min drive']

ValueError: too many values to unpack (expected 2)

In [83]:
branchLocations.loc[0,'15 min drive'] = 'hello'

In [17]:
api_response =  api.get_data((branchLocations.loc[0,'LATITUDE'],branchLocations.loc[0,'LONGITUDE']), 15)

branchLocations.loc[0,'geometry'] = shape(api_response)

0     POLYGON ((-87.76461 42.09724, -87.76462 42.095...
1                                                  None
2                                                  None
3                                                  None
4                                                  None
                            ...                        
77                                                 None
78                                                 None
79                                                 None
80                                                 None
81                                                 None
Name: geometry, Length: 82, dtype: geometry

In [88]:
branchLocations.loc[0,'15 min drive'] = []

ValueError: Must have equal len keys and value when setting with an iterable

In [68]:
branchLocations.loc[0,'15 min drive'] = branchLocations.loc[0,'15 min drive'].apply(api.get_data((branchLocations.loc[0,'LATITUDE'],
                                                                                                  branchLocations.loc[0,'LONGITUDE']), 15))

AttributeError: 'numpy.int64' object has no attribute 'apply'

In [69]:
branchLocations.loc[0,'LOCATION'] = branchLocations.loc[0,'LOCATION'].apply(api.get_data((branchLocations.loc[0,'LATITUDE'],
                                                                                          branchLocations.loc[0,'LONGITUDE']), 15))

AttributeError: 'str' object has no attribute 'apply'

In [ ]:
branchLocations[]

In [60]:
for i in range(0, len(branchLocations['15 min drive'])):
    branchLocations.loc[i,'15 min drive'] = api.get_data(branchLocations.loc[i,'LOCATION'], 15)

ValueError: too many values to unpack (expected 2)

In [57]:
branchLocations['15 min drive'] = branchLocations['15 min drive'].apply(lambda x: api.get_data(branchLocations['LOCATION'], 15))


ValueError: too many values to unpack (expected 2)

In [32]:
branchLocations.head()

,BRANCH,HOURS OF OPERATION,ADDRESS,CITY,STATE,ZIP,PHONE,WEBSITE,PERMANENTLY_CLOSED,LATITUDE,LONGITUDE,15 min drive
0,Albany Park,"Mon. & Wed., 10-6; Tues. & Thurs., Noon-8; Fri...",3401 W. Foster Ave.,Chicago,IL,60625,(773) 539-5450,https://www.chipublib.org/locations/3/,0,41.975579,-87.713613,"[5, 6, 7]"
1,Altgeld,"Mon. & Wed., Noon-8; Tues. & Thurs., 10-6; Fri...",955 E. 131st Street,Chicago,IL,60827,(312) 747-3270,https://www.chipublib.org/locations/4/,0,41.657198,-87.598838,"[5, 6, 7]"
2,Archer Heights,"Mon. & Wed., Noon-8; Tues. & Thurs., 10-6; Fri...",5055 S. Archer Ave.,Chicago,IL,60632,(312) 747-9241,https://www.chipublib.org/locations/5/,0,41.801108,-87.726484,"[5, 6, 7]"
3,Austin,"Mon. & Wed., Noon-8; Tues. & Thurs., 10-6; Fri...",5615 W. Race Ave.,Chicago,IL,60644,(312) 746-5038,https://www.chipublib.org/locations/6/,0,41.889173,-87.765820,"[5, 6, 7]"
4,Austin-Irving,"Mon. & Wed., Noon-8; Tues. & Thurs., 10-6; Fri...",6100 W. Irving Park Rd.,Chicago,IL,60634,(312) 744-6222,https://www.chipublib.org/locations/7/,0,41.953123,-87.779285,"[5, 6, 7]"
